<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/IT_EFAS_MACRO_ECON_DB/IT_EFAS_MACRO_ECON_DB_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import pandas as pd
from datetime import date

today = date.today()

##### 조회년월
queryYYYYMM = '202212'
# queryYYYYMM = str(today.year) + str(today.month)

##### 조회분기
queryYYYYQQ = '2022Q3'
# queryYYYYQQ = str(today.year) + ('Q1' if today.month <= 3 else 'Q2' if today.month <= 6 else 'Q3' if today.month <= 9 else 'Q4')

key = 'MTSA07N58C5X8EB4LMJL'

def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'])
  df['STD_YM'] = df['STD_YM'].str.replace('Q1$', '03', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q2$', '06', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q3$', '09', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q4$', '12', regex=True)
  return df

# (1) GDP(원계열, 실질)
# 2.1.2.1.4. 경제활동별 GDP 및 GNI(원계열, 실질, 분기) : [200Y006][Q]   /   국내총생산(시장가격, GDP) : [1400][십억원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y006/Q/2010Q1/2020Q4/1400
GDP_WON_SIL = ecosApiCall(f"200Y006/Q/2010Q1/{queryYYYYQQ}/1400")
GDP_WON_SIL.rename(columns = {'DATA_VALUE':'GDP_WON_SIL'}, inplace = True)

# (2) GDP(원계열, 명목)
# 2.1.2.1.3. 경제활동별 GDP 및 GNI(원계열, 명목, 분기) : [200Y005][Q]   /   국내총생산(시장가격, GDP) : [1400][십억원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y005/Q/2010Q1/2020Q4/1400
GDP_WON_MYUNG = ecosApiCall(f"200Y005/Q/2010Q1/{queryYYYYQQ}/1400")
GDP_WON_MYUNG.rename(columns = {'DATA_VALUE':'GDP_WON_MYUNG'}, inplace = True)

# (3) GDP(계절조정, 실질)
# 2.1.2.1.2. 경제활동별 GDP 및 GNI(계절조정, 실질, 분기) : [200Y004][Q]   /   국내총생산(시장가격, GDP) : [1400][십억원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y004/Q/2010Q1/2020Q4/1400
GDP_GAE_SIL = ecosApiCall(f"200Y004/Q/2010Q1/{queryYYYYQQ}/1400")
GDP_GAE_SIL.rename(columns = {'DATA_VALUE':'GDP_GAE_SIL'}, inplace = True)

# (4) GDP(계절조정, 명목)
# 2.1.2.1.1. 경제활동별 GDP 및 GNI(계절조정, 명목, 분기) : [200Y003][Q]   /   국내총생산(시장가격, GDP) : [1400][십억원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y004/Q/2010Q1/2020Q4/1400
GDP_GAE_MYUNG = ecosApiCall(f"200Y003/Q/2010Q1/{queryYYYYQQ}/1400")
GDP_GAE_MYUNG.rename(columns = {'DATA_VALUE':'GDP_GAE_MYUNG'}, inplace = True)

# (5) GDP 성장률(원계열, 실질) - 전년동기비
# 2.1.1.2. 주요지표(분기지표) : [200Y002][Q]   /   국내총생산(GDP)(실질, 원계열, 전년동기비) [10211][%]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y002/Q/2010Q1/2020Q4/10211
INDU_GDP_WON = ecosApiCall(f"200Y002/Q/2010Q1/{queryYYYYQQ}/10211")
INDU_GDP_WON.rename(columns = {'DATA_VALUE':'INDU_GDP_WON'}, inplace = True)

# (6) GDP 성장률(계절조정, 실질) - 전기비
# 2.1.1.2. 주요지표(분기지표) : [200Y002][Q]   /   국내총생산(GDP)(실질, 계절조정, 전기비) [10111][%]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/200Y002/Q/2010Q1/2020Q4/10111
INDU_GDP_GAE = ecosApiCall(f"200Y002/Q/2010Q1/{queryYYYYQQ}/10111")
INDU_GDP_GAE.rename(columns = {'DATA_VALUE':'INDU_GDP_GAE'}, inplace = True)

# (7) 전산업생산지수_농림어업제외(원계열)
# 8.1.4. 전산업생산지수(농림어업제외) : [901Y033][M]  /  전산업생산지수(농림어업 제외) : [A00][2015=100]  /  원계열 : [1]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y033/M/201001/202209/A00/1
INDU_OPERATING_WON = ecosApiCall(f"901Y033/M/201001/{queryYYYYMM}/A00/1")
INDU_OPERATING_WON.rename(columns = {'DATA_VALUE':'INDU_OPERATING_WON'}, inplace = True)

# (8) 전산업생산지수_농림어업제외(계절조정)
# 8.1.4. 전산업생산지수(농림어업제외) : [901Y033][M]  /  전산업생산지수(농림어업 제외) : [A00][2015=100]  /  계절조정 : [2]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y033/M/201001/202209/A00/2
INDU_OPERATING_GAE = ecosApiCall(f"901Y033/M/201001/{queryYYYYMM}/A00/2")
INDU_OPERATING_GAE.rename(columns = {'DATA_VALUE':'INDU_OPERATING_GAE'}, inplace = True)

# (9) 제조업 생산지수(원계열)
# 8.3.1. 광업/제조업 - 산업별 생산/출하/재고 지수 : [901Y032][M]  /  제조업 : [I11AC][2015=100]  /  생산지수(원지수) : [1]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y032/M/201001/202209/I11AC/1
INDU_MANU_OPER_WON = ecosApiCall(f"901Y032/M/201001/{queryYYYYMM}/I11AC/1")
INDU_MANU_OPER_WON.rename(columns = {'DATA_VALUE':'INDU_MANU_OPER_WON'}, inplace = True)

# (10) 제조업 생산지수(계절조정)
# 8.3.1. 광업/제조업 - 산업별 생산/출하/재고 지수 : [901Y032][M]  /  제조업 : [I11AC][2015=100]  /  생산지수(계절조정) : [2]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y032/M/201001/202209/I11AC/2
INDU_MANU_OPER_GAE = ecosApiCall(f"901Y032/M/201001/{queryYYYYMM}/I11AC/2")
INDU_MANU_OPER_GAE.rename(columns = {'DATA_VALUE':'INDU_MANU_OPER_GAE'}, inplace = True)

# (11) 서비스업 생산지수(경상지수)
# 8.5.1. 서비스업 - 산업별 서비스업생산지수 : [901Y038][M]  /  총지수 : [I51A][2015=100]  /  경상지수 : [1]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y038/M/201001/202209/I51A/1
INDU_SERV_OPER_GYUNG = ecosApiCall(f"901Y038/M/201001/{queryYYYYMM}/I51A/1")
INDU_SERV_OPER_GYUNG.rename(columns = {'DATA_VALUE':'INDU_SERV_OPER_GYUNG'}, inplace = True)

# (12) 서비스업 생산지수(불변지수)
# 8.5.1. 서비스업 - 산업별 서비스업생산지수 : [901Y038][M]  /  총지수 : [I51A][2015=100]  /  불변지수 : [2]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y038/M/201001/202209/I51A/2
INDU_SERV_OPER_BUL = ecosApiCall(f"901Y038/M/201001/{queryYYYYMM}/I51A/2")
INDU_SERV_OPER_BUL.rename(columns = {'DATA_VALUE':'INDU_SERV_OPER_BUL'}, inplace = True)

# (13) 서비스업 생산지수(계절조정지수)
# 8.5.1. 서비스업 - 산업별 서비스업생산지수 : [901Y038][M]  /  총지수 : [I51A][2015=100]  /  계절조정지수 : [3]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y038/M/201001/202209/I51A/3
INDU_SERV_OPER_GAE = ecosApiCall(f"901Y038/M/201001/{queryYYYYMM}/I51A/3")
INDU_SERV_OPER_GAE.rename(columns = {'DATA_VALUE':'INDU_SERV_OPER_GAE'}, inplace = True)

# (14) 소매업태별 판매액지수(경상지수)
# 8.5.3. 서비스업 - 소매업태별 판매액지수 : [901Y098][M]  /  총지수 : [I74A][2015=100]  /  경상지수 : [I74A]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y098/M/201001/202209/I74A/I74A
INDU_CONSALE_GYUNG = ecosApiCall(f"901Y098/M/201001/{queryYYYYMM}/I74A/I74A")
INDU_CONSALE_GYUNG.rename(columns = {'DATA_VALUE':'INDU_CONSALE_GYUNG'}, inplace = True)

# (15) 소매업태별 판매액지수(불변지수)
# 8.5.3. 서비스업 - 소매업태별 판매액지수 : [901Y098][M]  /  총지수 : [I74A][2015=100]  /  불변지수 : [I74B]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y098/M/201001/202209/I74B/I74B
INDU_CONSALE_BUL = ecosApiCall(f"901Y098/M/201001/{queryYYYYMM}/I74B/I74B")
INDU_CONSALE_BUL.rename(columns = {'DATA_VALUE':'INDU_CONSALE_BUL'}, inplace = True)

# (16) 소매업태별 판매액지수(계절조정지수)
# 8.5.3. 서비스업 - 소매업태별 판매액지수 : [901Y098][M]  /  총지수 : [I74A][2015=100]  /  계절조정지수 : [I74C]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y098/M/201001/202209/I74C/I74C
INDU_CONSALE_GAE = ecosApiCall(f"901Y098/M/201001/{queryYYYYMM}/I74C/I74C")
INDU_CONSALE_GAE.rename(columns = {'DATA_VALUE':'INDU_CONSALE_GAE'}, inplace = True)

# (17) 원달러 환율
# 3.1.2.1. 주요국 통화의 대원화환율 : [731Y004][M]   /   원/미국달러(매매기준율) : [0000001][원]   /   평균자료 : [0000100]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/731Y004/M/201001/202208/0000001/0000100
EXCHANGE_WON_DOL = ecosApiCall(f"731Y004/M/201001/{queryYYYYMM}/0000001/0000100")
EXCHANGE_WON_DOL.rename(columns = {'DATA_VALUE':'EXCHANGE_WON_DOL'}, inplace = True)

# (18) 시장금리 (KORIBOR 3개월)
# 1.3.2.2 시장금리(월,분기,년) : [721Y001][M]   /   KORIBOR(3개월) : [1025000][연%]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/721Y001/M/201001/202208/1025000
INTERATE_RATE_KORIBOR3 = ecosApiCall(f"721Y001/M/201001/{queryYYYYMM}/1025000")
INTERATE_RATE_KORIBOR3.rename(columns = {'DATA_VALUE':'INTERATE_RATE_KORIBOR3'}, inplace = True)

# (19) 물가 - 생산자 물가지수(에너지, 식료품 제외) (산연)
# 4.1.1.2. 생산자물가지수(특수분류) : [404Y015][M]   /   식료품에너지이외 : [S520AA][2015=100]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/404Y015/M/201001/202208/S520AA
MANU_PRICE = ecosApiCall(f"404Y015/M/201001/{queryYYYYMM}/S520AA")
MANU_PRICE.rename(columns = {'DATA_VALUE':'MANU_PRICE'}, inplace = True)

# (20) 유가(Dubai)
# 9.1.6.3. 국제상품가격 : [902Y003][M]   /   Dubai(현물) : [4010102][U$/bbl]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/902Y003/M/201001/202208/4010102
OIL_PRICE_DUBAI = ecosApiCall(f"902Y003/M/201001/{queryYYYYMM}/4010102")
OIL_PRICE_DUBAI.rename(columns = {'DATA_VALUE':'OIL_PRICE_DUBAI'}, inplace = True)


In [19]:
# compile the list of dataframes you want to merge such as [GDP_WON, GDP_GAE, OIL_PRICE_DUBAI ... ]
df_list = [GDP_WON_SIL, GDP_WON_MYUNG, GDP_GAE_SIL, GDP_GAE_MYUNG, INDU_GDP_WON, INDU_GDP_GAE, INDU_OPERATING_WON, INDU_OPERATING_GAE, INDU_MANU_OPER_WON, INDU_MANU_OPER_GAE, INDU_SERV_OPER_GYUNG, INDU_SERV_OPER_BUL, INDU_SERV_OPER_GAE, INDU_CONSALE_GYUNG, INDU_CONSALE_BUL, INDU_CONSALE_GAE, EXCHANGE_WON_DOL, INTERATE_RATE_KORIBOR3, MANU_PRICE, OIL_PRICE_DUBAI]
# grab first dataframe
IT_EFAS_MACRO_ECON_DB = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    IT_EFAS_MACRO_ECON_DB = pd.merge(left = IT_EFAS_MACRO_ECON_DB, right = to_merge, left_on = 'STD_YM', right_on = 'STD_YM', how = "outer")

IT_EFAS_MACRO_ECON_DB = IT_EFAS_MACRO_ECON_DB.set_index('STD_YM')

today = date.today()
IT_EFAS_MACRO_ECON_DB.to_excel(f'{today}_IT_EFAS_MACRO_ECON_DB.xlsx')
IT_EFAS_MACRO_ECON_DB

,GDP_WON_SIL,GDP_WON_MYUNG,GDP_GAE_SIL,GDP_GAE_MYUNG,INDU_GDP_WON,INDU_GDP_GAE,INDU_OPERATING_WON,INDU_OPERATING_GAE,INDU_MANU_OPER_WON,INDU_MANU_OPER_GAE,INDU_SERV_OPER_GYUNG,INDU_SERV_OPER_BUL,INDU_SERV_OPER_GAE,INDU_CONSALE_GYUNG,INDU_CONSALE_BUL,INDU_CONSALE_GAE,EXCHANGE_WON_DOL,INTERATE_RATE_KORIBOR3,MANU_PRICE,OIL_PRICE_DUBAI
STD_YM,,,,,,,,,,,,,,,,,,,,
201003,335204.8,307737.9,348541.7,320326.5,7.3,2.0,94.0,91.0,93.715,90.078,84.4,90.7,89.1,80.9,93.9,104.7,1137.64,2.82,97.90,79.05
201006,356501.1,329481.0,355163.3,329462.5,7.9,1.9,94.3,92.1,95.102,92.427,85.1,90.9,89.8,81.3,90.4,106.0,1212.33,2.46,99.96,73.24
201009,357293.6,334833.6,359266.2,333669.4,5.8,1.2,89.8,90.7,90.573,93.069,83.5,88.6,89.0,86.1,98.9,107.8,1167.01,2.68,100.61,77.32
201012,377618.5,350558.7,363646.8,339152.8,6.3,1.2,102.4,92.5,98.983,95.914,92.7,98.0,90.3,91.6,120.8,107.6,1147.55,2.80,102.24,88.54
201103,353395.6,328096.0,367040.1,340957.3,5.4,0.9,97.6,94.2,102.126,97.978,89.4,93.0,91.4,89.8,103.5,106.9,1122.45,3.31,105.29,109.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202207,NaN,NaN,NaN,NaN,NaN,NaN,116.1,117.8,118.500,118.200,134.7,116.1,116.7,135.5,114.9,85.2,1307.40,2.51,117.56,105.80
202208,NaN,NaN,NaN,NaN,NaN,NaN,113.9,117.7,112.200,116.700,135.3,116.6,118.8,136.2,107.9,92.7,1318.44,2.70,117.05,95.55
202210,NaN,NaN,NaN,NaN,NaN,NaN,115.4,115.2,114.100,110.400,135.8,117.2,117.4,141.0,129.9,88.8,1426.66,3.52,117.25,90.04


In [16]:
INDU_SERV_OPER_GYUNG

,STD_YM,INDU_SERV_OPER_GYUNG
0,201001,78.8
1,201002,77.8
2,201003,84.4
3,201004,82.7
4,201005,84.7
...,...,...
150,202207,134.7
151,202208,135.3
152,202209,135.6
153,202210,135.8
